<DIV ALIGN=CENTER>

# Introduction to Deep Learning
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

Text

-----

-----
### Student Activity

In the preceding cells, we introduced Spark. Now that you have run the Notebook, go back and make the
following changes to see how the results change.

1. Change the `myRDD` example to start with all integers from 0 to 199.
Use an appropriate lambda function to convert this RDD to a new RDD that
has all odd integers from 1 to 399.

2. Filter the previous RDD to contain only entries that are divisible by
9.

3. Convert this RDD to a Spark DataFrame, specify the column name as
`Numbers`.

4. Add an index column to this Spark DataFrame, which sequentially
increases.

Additional, more advanced problems:

1. Create an RDD containing the 'Year', 'Month', 'DayofMonth', 'dDelay',
and 'Origin' columns for the airline data..

2. Filter this RDD to contain only flight data for flights leaving O'Hare
airport.

-----